In [1]:
import pandas as pd
import numpy as np
import re
import torch
import os
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from skimage import io
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer #Pretrained model for text embedding
from torchviz import make_dot
from tqdm import tqdm

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
def read_data(fname):
    return pd.read_csv(fname,index_col=0)


def clean_text(text):
    return re.sub(r'[^\w\s]', '',text).lower()


def Preprocessing(df):
    null_cols=df.isnull().sum()*100/len(df)
    null_cols=null_cols[null_cols!=0]
    
    #Replacing NaNs
    for idx,val in null_cols.items():
        if df.dtypes[idx] == object: #replacing with mode if object (i-e string etc)
            df[idx].fillna(df[idx].mode()[0],inplace=True)        
        else: #replacing with average if integer,float etc
            df[idx].fillna(df[idx].mean(),inplace=True)
     
    #Encoding to numeric values
    alpha_cols=df.select_dtypes(include=['object'])
    try:
        df['overall_sentiment']= df['overall_sentiment'].map({'neutral': 0, 'positive': 1,'negative':2})      
    except:
        pass
    
    #Cleaning text
    for i in range(len(df)):
        try:
            df1['text_corrected'][i] = clean_text(df1['text_corrected'][i])
        except:
            pass
def Preprocessingtest(df):
    null_cols=df.isnull().sum()*100/len(df)
    null_cols=null_cols[null_cols!=0]
    
    #Replacing NaNs
    for idx,val in null_cols.items():
        if df.dtypes[idx] == object: #replacing with mode if object (i-e string etc)
            df[idx].fillna(df[idx].mode()[0],inplace=True)        
        else: #replacing with average if integer,float etc
            df[idx].fillna(df[idx].mean(),inplace=True)
     
    #Encoding to numeric values
    alpha_cols=df.select_dtypes(include=['object'])
    #Cleaning text
    for i in range(len(df)):
        try:
            df1['text_corrected'][i] = clean_text(df1['text_corrected'][i])
        except:
            pass
        
    
def get_max_length_words(df,col):
    try:
        return df[col].str.split("\\s+").str.len().max()
    except:
        RaiseError("Invalid Column passed")
        
        
def padding(tensor,length):
    tensor=torch.tensor(tensor) if not torch.is_tensor(tensor) else tensor
    return torch.nn.functional.pad(tensor,(0, length-tensor.size()[0]))

In [4]:
class MemeDataset(Dataset):
    def __init__(self, dataframe, root_dir, pad_length,model,transform = None, freq_threshold = 5):
        self.df = dataframe.copy()
        self.img = self.df['image_name']
        self.pad_len=pad_length
        self.captions = self.df['text_corrected']
        self.root_dir = root_dir
        self.transform = transform
        self.model=model

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        #convert image to 3 channel RGB, as some images in our dataset are >3 channels or <3 channels
        image = Image.open(os.path.join(self.root_dir, self.df.iloc[index, 0])).convert("RGB")
        y_label = torch.tensor(int(self.df.iloc[index, 2]))
        caption = self.captions[index]
        if self.transform:
            image = self.transform(image)   
        padded_text_tensor = padding(self.model.encode(caption,convert_to_tensor=True),self.pad_len)
        return (image, padded_text_tensor, y_label)

In [5]:
class NN_img(nn.Module):
    """
    Network for images
    """
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Flatten(),
            nn.Linear(100*100*3, 1024),
            nn.ReLU(),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,60),
            nn.Tanh(),
            nn.Linear(60,3)
        )
            
    def forward(self, x):
        return self.network(x)

    
class NN_text(nn.Module):
    """
    Network for text
    """
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
        nn.Flatten(),
        nn.Linear(200, 160),
        nn.ReLU(),
        nn.Linear(160,128),
        nn.ReLU(),
        nn.Linear(128,72),
        nn.Tanh(),
        nn.Linear(72,64),
        nn.Tanh(),
        nn.Linear(64,3)
        )

    def forward(self, x):
        return self.network(x)

    
class NN_combined(nn.Module):
    """
    Combined network that uses both image and text network
    """
    def __init__(self, modelA, modelB):
        super().__init__()
        self.modelA = modelA
        self.modelB = modelB
        
        self.network = nn.Sequential(
        nn.Linear(6,64),
        nn.ReLU(),
        nn.Linear(64,32),
        nn.ReLU(),
        nn.Linear(32,16),
        nn.ReLU(),
        nn.Linear(16,6),
        nn.ReLU(),
        nn.Linear(6, 3)
        )
        
    def forward(self, x1, x2):
        x1 = self.modelA(x1)
        x2 = self.modelB(x2)
        x = torch.cat((x1, x2), dim=1)
        return self.network(x)
    
    
def __train__(dataloader, model, loss_fn, optimizer):
    """
    Trains network on trainset
    """
    size = len(dataloader.dataset)
    for batch, (img, txt, y) in enumerate(dataloader):
        img=img.to(device)
        txt=txt.to(device)
        y=y.to(device)
        pred = model(img,txt)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss, current = loss.item(), batch*len(img)
        print(f"loss: {loss:>2f} [{current:>5d}/{size:>2d}]")


def __valid__(dataloader, model, loss_fn,acc_list):
    """
    Performs evaluation on testset
    """
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for img, txt, y in dataloader:
            img=img.to(device)
            txt=txt.to(device)
            y=y.to(device)
            pred = model(img,txt)
            y_hat = pred.argmax(dim=1).cpu().numpy()
            pred_txt = [str(w) for w in y_hat]
            print(pred_txt)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss/=num_batches
    correct/=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.2f}%, Average loss: {test_loss:>2f} \n")
    acc_list.append(100*correct)

In [6]:
# 导入os模块
import os
from pandas import DataFrame
path = "C:\\Users\\86187\\AI\\finalproject\\data"
data={'image_name':[],
     'text_corrected':[],
     'overall_sentiment':[]}
number=[]
sentiment=[]
# os.listdir()方法获取文件夹名字，返回数组
with open("C:\\Users\\86187\\AI\\finalproject\\train.txt",'r') as f:
        for line in f:
            s=str(line).strip('\n').split(',')
            number.append(s[0])
            sentiment.append(s[1])
        f.close()
del(number[0])
del(sentiment[0])
file_name_list = os.listdir(path)
img=[]
txt=[]
for i in range(len(file_name_list)):
    if(i%2==0):
        img.append(file_name_list[i])
    else:
        txt.append(file_name_list[i])
for i in range(len(img)):
    z=str(img[i]).split('.')
    if z[0] in number:
        data['image_name'].append(file_name_list[2*i])
        with open("C:\\Users\\86187\\AI\\finalproject\\data\\"+str(txt[i]),'r',encoding="ANSI") as f:
            for line in f:
                data['text_corrected'].append(line)
            f.close()
            data['overall_sentiment'].append(sentiment[number.index(z[0])])
df1=pd.DataFrame(data)
df1

,image_name,text_corrected,overall_sentiment
0,1.jpg,How I feel today #legday #jelly #aching #gym \n,positive
1,10.jpg,@ArrivaTW absolute disgrace two carriages from...,negative
2,100.jpg,This is my Valentine's from 1 of my nephews. I...,positive
3,1001.jpg,Zoe's first love #Rattled @JohnnyHarper15 \n,positive
4,1002.jpg,Chaotic Love - giclee print ?65 at #art #love ...,positive
...,...,...,...
3995,995.jpg,OMG. Well done #Eskom! 'Man dies during #LoadS...,positive
3996,996.jpg,Feelin' the love in here! #ValentinesDay #cari...,positive
3997,997.jpg,#blue #eyes can't be #beaten \n,positive
3998,998.jpg,LA CHUCHA LOUUU TE CHUPO LOS OJOS..! \n,positive


In [7]:
data1={'image_name':[],
     'text_corrected':[],
     'overall_sentiment':[]}
number1=[]
sentiment1=[]
with open("C:\\Users\\86187\\AI\\finalproject\\test_without_label.txt",'r') as f:
        for line in f:
            s=str(line).strip('\n').split(',')
            number1.append(s[0])
            sentiment1.append(s[1])
        f.close()
del(number1[0])
del(sentiment1[0])
file_name_list = os.listdir(path)
img1=[]
txt1=[]
label=[]
for i in range(len(file_name_list)):
    if(i%2==0):
        img1.append(file_name_list[i])
    else:
        txt1.append(file_name_list[i])
for i in range(len(img1)):
    z=str(img1[i]).split('.')    
    if z[0] in number1:
        data1['image_name'].append(file_name_list[2*i])
        with open("C:\\Users\\86187\\AI\\finalproject\\data\\"+str(txt1[i]),'r',encoding="ANSI") as f:
            for line in f:
                data1['text_corrected'].append(line)
            f.close()
            data1['overall_sentiment'].append(z[0])
            label.append(z[0])
df2=pd.DataFrame(data1)
df2

,image_name,text_corrected,overall_sentiment
0,1000.jpg,betterfeelingfilms: RT via Instagram: First da...,1000
1,1020.jpg,RT @GooleAFC: Tonight we recieved a donation o...,1020
2,1021.jpg,#SPEECHLESS. Deah & Yusor were just married in...,1021
3,1025.jpg,RT @AstroTerry: #speechless from this #sunrise \n,1025
4,1036.jpg,How I feel after I #lift! The #pump is #strong...,1036
...,...,...,...
506,948.jpg,#vsco #vscocam #vscohub #vscobest #vscoonly #v...,948
507,96.jpg,Picture Frame #Distressed Red Black #Sentiment...,96
508,960.jpg,RT @FriendsElderly: Loneliness is as big a ris...,960
509,964.jpg,Is the suspense killing you? Cause I'm pretty ...,964


In [8]:
Preprocessing(df1) #Cleaning NaNs, text, encoding sentiment to numeric values
Preprocessingtest(df2)
df1
df2

,image_name,text_corrected,overall_sentiment
0,1000.jpg,betterfeelingfilms: RT via Instagram: First da...,1000
1,1020.jpg,RT @GooleAFC: Tonight we recieved a donation o...,1020
2,1021.jpg,#SPEECHLESS. Deah & Yusor were just married in...,1021
3,1025.jpg,RT @AstroTerry: #speechless from this #sunrise \n,1025
4,1036.jpg,How I feel after I #lift! The #pump is #strong...,1036
...,...,...,...
506,948.jpg,#vsco #vscocam #vscohub #vscobest #vscoonly #v...,948
507,96.jpg,Picture Frame #Distressed Red Black #Sentiment...,96
508,960.jpg,RT @FriendsElderly: Loneliness is as big a ris...,960
509,964.jpg,Is the suspense killing you? Cause I'm pretty ...,964


In [9]:
alpha_len=170 #arbitrary value to add for padding on top of max length sentence
alpha_len1=173
pad_len=get_max_length_words(df1,'text_corrected')+alpha_len
pad_len1=get_max_length_words(df2,'text_corrected')+alpha_len1
df1

,image_name,text_corrected,overall_sentiment
0,1.jpg,how i feel today legday jelly aching gym \n,1
1,10.jpg,arrivatw absolute disgrace two carriages from ...,2
2,100.jpg,this is my valentines from 1 of my nephews i a...,1
3,1001.jpg,zoes first love rattled johnnyharper15 \n,1
4,1002.jpg,chaotic love giclee print 65 at art love chao...,1
...,...,...,...
3995,995.jpg,omg well done eskom man dies during loadsheddi...,1
3996,996.jpg,feelin the love in here valentinesday caring \n,1
3997,997.jpg,blue eyes cant be beaten \n,1
3998,998.jpg,la chucha louuu te chupo los ojos \n,1


In [10]:
vector_model=SentenceTransformer('all-MiniLM-L6-v2').to(device) #Pretrained model to generate text embeddings
#All credit to: https://www.sbert.net/docs/pretrained_models.html
dataset = MemeDataset(dataframe = df1, root_dir = './data', pad_length=pad_len,model=vector_model,transform = transforms.Compose(
    [
     transforms.Resize((100, 100)),
     transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
    ]))
test_set = MemeDataset(dataframe = df2, root_dir = './data', pad_length=pad_len1,model=vector_model,transform = transforms.Compose(
    [
     transforms.Resize((100, 100)),
     transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
    ]))

In [11]:
#Loading train and test sets
train_set, valid_set = torch.utils.data.random_split(dataset, [3000, 1000])
dataloader_train = DataLoader(dataset = train_set, batch_size = 32, shuffle=True)
dataloader_valid = DataLoader(dataset = valid_set, batch_size = 32, shuffle=True)
dataloader_test = DataLoader(dataset = test_set, batch_size = 32, shuffle=True)

In [12]:
#Setting up Neural Networks
net_img = NN_img().to(device) #image network
net_text = NN_text().to(device) #text network 
combined_model = NN_combined(net_img, net_text).to(device) #combined network

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)

epochs = 10
acc=list()
for t in tqdm(range(epochs)):
    print(f"Epoch {t+1}\n_______________________________")
    __train__(dataloader_train, combined_model, loss_fn, optimizer)
    __valid__(dataloader_valid, combined_model, loss_fn,acc)
    if(t>=2 and t%2==0):
            print(f"Accuracy of t-3 iter: {acc[t-2]}\nAccuracy of current iter: {acc[t]}")
print(f"Highest accuracy was: {max(acc)}")
result=[]
numberlabel=[]
def __test__(dataloader, model,label):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    res=[]
    number=[]

    with torch.no_grad():
        for img, txt, y in dataloader:
            img=img.to(device)
            txt=txt.to(device)
            y=y.to(device)
            pred = model(img,txt)
            y_hat = pred.argmax(dim=1).cpu().numpy()
            pred_txt = [str(w) for w in y_hat]
            print(pred_txt)
            print(y.cpu().numpy())
            res.append(pred_txt)
            number.append(y.cpu().numpy())
    return res,number
result,numberlabel=__test__(dataloader_test, combined_model,label)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Epoch 1
_______________________________
loss: 1.179351 [    0/3000]
loss: 1.195014 [   32/3000]
loss: 1.165962 [   64/3000]
loss: 1.064730 [   96/3000]
loss: 1.165483 [  128/3000]
loss: 1.147674 [  160/3000]
loss: 1.124826 [  192/3000]
loss: 1.074771 [  224/3000]
loss: 1.137060 [  256/3000]
loss: 1.028588 [  288/3000]
loss: 1.070834 [  320/3000]
loss: 1.135008 [  352/3000]
loss: 1.145807 [  384/3000]
loss: 1.175571 [  416/3000]
loss: 1.092737 [  448/3000]
loss: 1.110064 [  480/3000]
loss: 1.072515 [  512/3000]
loss: 1.058111 [  544/3000]
loss: 1.091250 [  576/3000]
loss: 1.068023 [  608/3000]
loss: 1.066875 [  640/3000]
loss: 1.070753 [  672/3000]
loss: 0.973510 [  704/3000]
loss: 0.996798 [  736/3000]
loss: 0.986979 [  768/3000]
loss: 0.940919 [  800/3000]
loss: 1.011896 [  832/3000]
loss: 0.923099 [  864/3000]
loss: 1.063856 [  896/3000]
loss: 0.893944 [  928/3000]
loss: 0.793884 [  960/3000]
loss: 0.812441 [  992/3000]
loss: 0.856238 [ 1024/3000]
loss: 0.963620 [ 1056/3000]
loss: 0.

 10%|████████▎                                                                          | 1/10 [01:12<10:54, 72.74s/it]

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
['1', '1', '1', '1', '1', '1', '1', '1']
Test Error: 
 Accuracy: 59.20%, Average loss: 0.906914 

Epoch 2
_______________________________
loss: 0.925202 [    0/3000]
loss: 0.760656 [   32/3000]
loss: 0.880520 [   64/3000]
loss: 0.941832 [   96/3000]
loss: 0.791240 [  128/3000]
loss: 0.922935 [  160/3000]
loss: 1.021747 [  192/3000]
loss: 0.894123 [  224/3000]
loss: 0.830243 [  256/3000]
loss: 1.047088 [  288/3000]
loss: 0.953892 [  320/3000]
loss: 0.899445 [  352/3000]
loss: 0.927772 [  384/3000]
loss: 0.843215 [  416/3000]
loss: 0.846272 [  448/3000]
loss: 0.804324 [  480/3000]
loss: 0.880500 [  512/3000]
loss: 0.902057 [  544/3000]
loss: 0.717515 [  576/3000]
loss: 0.893988 [  608/3000]
loss: 1.123333 [  640/3000]
loss: 0.769893 [  672/3000]
loss: 0.684139 [  704/3000]
loss: 0.748383 [  736/3000]
loss: 0.753995 [  768/3000]
l

 20%|████████████████▌                                                                  | 2/10 [02:25<09:40, 72.51s/it]

...

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [11:38<01:20, 80.96s/it]

['1', '1', '1', '1', '1', '1', '1', '2', '2', '1', '2', '1', '2', '1', '2', '2', '1', '1', '1', '2', '2', '2', '0', '2', '1', '1', '1', '2', '1', '1', '1', '2']
['1', '1', '1', '2', '1', '1', '1', '2']
Test Error: 
 Accuracy: 61.20%, Average loss: 1.099638 

Accuracy of t-3 iter: 61.8
Accuracy of current iter: 61.199999999999996
Epoch 10
_______________________________
loss: 0.568028 [    0/3000]
loss: 0.246271 [   32/3000]
loss: 0.269477 [   64/3000]
loss: 0.508578 [   96/3000]
loss: 0.233289 [  128/3000]
loss: 0.301550 [  160/3000]
loss: 0.368519 [  192/3000]
loss: 0.433825 [  224/3000]
loss: 0.583905 [  256/3000]
loss: 0.307159 [  288/3000]
loss: 0.363069 [  320/3000]
loss: 0.549557 [  352/3000]
loss: 0.363346 [  384/3000]
loss: 0.338247 [  416/3000]
loss: 0.564573 [  448/3000]
loss: 0.393043 [  480/3000]
loss: 0.134818 [  512/3000]
loss: 0.559168 [  544/3000]
loss: 0.558565 [  576/3000]
loss: 0.386273 [  608/3000]
loss: 0.476480 [  640/3000]
loss: 0.302913 [  672/3000]
loss: 0.3949

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [12:36<00:00, 75.68s/it]

['1', '1', '1', '1', '1', '2', '2', '1', '1', '1', '0', '2', '2', '0', '1', '2', '2', '0', '2', '1', '1', '1', '0', '2', '2', '0', '1', '1', '1', '1', '1', '1']
['1', '1', '2', '1', '0', '0', '1', '0']
Test Error: 
 Accuracy: 62.10%, Average loss: 1.292449 

Highest accuracy was: 66.5


['1', '1', '2', '2', '1', '1', '1', '1', '2', '0', '2', '0', '0', '1', '1', '1', '1', '1', '2', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
[4201 1021  558  310 4287 3336  948 2717 2635 1061   55 4434  780  184
  145 2362 4378 4925 4138  272   58  916 4678 3005 3374 3745 1999 3348
 3842 4319 4877 3641]
['2', '1', '1', '1', '0', '0', '0', '0', '1', '1', '2', '1', '1', '1', '1', '1', '2', '1', '2', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1']
[4944 3139 4417 5111 4981 1660  443 2962 1227  124 4345  740 3891 3206
   13 1036 3774  600 2029 2792 3126 3714 2046 3495 3346 3570 3243  173
 3022 1048 2758 1949]
['1', '0', '1', '0', '1', '1', '1', '1', '2', '1', '1', '1', '1', '0', '2', '0', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '2', '1', '1', '1', '2', '1']
[4288 4306  573 1516    5 3195 2704 1612 3889 3572 3003 5000 4587  964
 4710 3008 3671 1025 2750 3476 1039 3103 4796 1879 3292    8 1576 2709
 1393  812  867 4500]
['1', '0', '1', '0', '1',

In [28]:
newlabel=[]
file_data = ""
newres=[]
for i in range(len(numberlabel)):
    for j in range(len(numberlabel[i])):
        newlabel.append(str(numberlabel[i][j]))
        newres.append(str(result[i][j]))
with open("C:\\Users\\86187\\AI\\finalproject\\test_without_label.txt",'r') as f:
    for line in f:
        s=str(line).strip('\n').split(',')
        if s[0] in newlabel:
            line = line.replace('null',newres[newlabel.index(s[0])])
            file_data += line

In [29]:
with open("C:\\Users\\86187\\AI\\finalproject\\test_without_label.txt",'w') as f:
    f.write(file_data)